In [18]:
import pandas as pd 
from sklearn.model_selection import train_test_split


FILENAME = "train.csv"
df = pd.read_csv(FILENAME)



X = df.drop("Year", axis=1)
y = df["Year"]

nomi_colonne = list(X.columns)
print(nomi_colonne)

# Esegui lo split in set di addestramento, di test e di validazione
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=42, test_size=0.2)
train, val = train_test_split(train, random_state=42, test_size=0.2)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43', 'S44', 'S45', 'S46', 'S47', 'S48', 'S49', 'S50', 'S51', 'S52', 'S53', 'S54', 'S55', 'S56', 'S57', 'S58', 'S59', 'S60', 'S61', 'S62', 'S63', 'S64', 'S65', 'S66', 'S67', 'S68', 'S69', 'S70', 'S71', 'S72', 'S73', 'S74', 'S75', 'S76', 'S77', 'S78', 'S79', 'S80', 'S81', 'S82', 'S83', 'S84', 'S85', 'S86', 'S87', 'S88', 'S89']
Train Shape: (161392, 91) | Val Shape: (40348, 91) | Test Shape: (50435, 91)


In [19]:
from pytorch_tabular.models import TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

data_config = DataConfig(
    target=["Year"],
    continuous_cols= nomi_colonne,
    categorical_cols=[],
)
trainer_config = TrainerConfig(
    batch_size=2048,
    max_epochs=100,
)
optimizer_config = OptimizerConfig()
model_config = TabNetModelConfig(
    task="regression",
    gflu_stages=6,
    gflu_feature_init_sparsity=0.3,
    gflu_dropout=0.0,
    learning_rate=1e-3,
)

In [20]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=True
)

2024-02-08 14:57:03,671 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

In [21]:
tabular_model.fit(train=train, validation=val)

Seed set to 42


2024-02-08 14:57:03,704 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-02-08 14:57:03,731 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-02-08 14:57:05,327 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: GANDALFModel

2024-02-08 14:57:05,364 - {pytorch_tabular.models.gandalf.gandalf:109} - INFO - Data Aware Initialization of T0

...

Output()

/Users/acco/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 
'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

/Users/acco/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 
'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

2024-02-08 14:58:13,387 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-02-08 14:58:13,388 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

In [22]:
pred_df = tabular_model.predict(test)
pred_df.head()

,Year_prediction
90138,2000.240601
197151,2003.089844
100824,2003.551636
244781,2002.391846
70355,2003.914917


In [23]:
result = tabular_model.evaluate(test)

Output()

/Users/acco/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     69.4365005493164      │
│  test_mean_squared_error  │     69.4365005493164      │
└───────────────────────────┴───────────────────────────┘